In [1]:
import pandas as pd
import plotly.express as px
import warnings 
warnings.filterwarnings('ignore')

%matplotlib inline
# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import os

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('can_scores_quintiles_EN.csv')
df = df.rename(columns={'PRCDDA': 'poi_cbg'})
df['Type'] = 'Urban'
df.loc[df['Dissemination area (DA) Population'] < 400, 'Type'] = 'Rural'
df[df['Dissemination area (DA) Population'] >= 400]
df = df[['poi_cbg', 'Dissemination area (DA) Population', 'Type']]
df

,poi_cbg,Dissemination area (DA) Population,Type
0,10010165,539,Urban
1,10010166,264,Rural
2,10010167,341,Rural
3,10010168,549,Urban
4,10010169,296,Rural
...,...,...,...
54791,59590023,440,Urban
54792,59590024,318,Rural
54793,59590025,683,Urban
54794,59590026,583,Urban


In [33]:
data = pd.read_csv('safegraph.csv')
data1 = pd.read_csv('geometry.csv')
# drop rows with Nan values in raw_visit_counts

data = data.dropna(subset=['raw_visit_counts'])
data['mobility_score'] = data['normalized_visits_by_state_scaling'] * data['median_dwell'] / 60

In [67]:
data = data.dropna(subset=['latitude'])
data = data.dropna(subset=['longitude'])

In [38]:
data2 = data1[['latitude','longitude','placekey']]
data = data.merge(data2, on='placekey', how='left')

In [39]:
data

,placekey,parent_placekey,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,visitor_home_aggregation,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_week_brand,device_type,normalized_visits_by_state_scaling,normalized_visits_by_total_visits,normalized_visits_by_total_visitors,normalized_visits_by_region_naics_visits,normalized_visits_by_region_naics_visitors,mobility_score,latitude,longitude
0,222-222@5x6-k2d-nnq,NaN,Sunrise Senior Living,980 Lynn Valley Road,North Vancouver,BC,V7J 3V7,CA,SG_BRAND_516ba83a8efa5bf6bdc8bf2383baf9e1,Sunrise Senior Living,2020-09-07T00:00:00-07:00,2020-09-14T00:00:00-07:00,4,2,"[0,1,0,1,0,1,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",CA:59150148,{},{},"{""CA:59331457"":4}","{""CA"":4}",NaN,382.5,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":0,""61-120"":...",{},{},"{""android"":0,""ios"":0}",290.590368,3.052028e-06,3.187134e-06,0.005457,0.008989,1852.513598,49.332717,-123.047451
1,222-222@63v-mcd-z75,NaN,Shandell Homes,25 Shadybrook Cres,Guelph,ON,N1G 3G8,CA,NaN,NaN,2020-09-07T00:00:00-04:00,2020-09-14T00:00:00-04:00,8,1,"[0,1,2,0,1,2,2]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",CA:35230185,{},"{""CA:35230036"":4}",{},{},NaN,378.0,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":1,""61-120"":...",{},{},"{""android"":0,""ios"":0}",461.098543,1.925949e-06,4.801766e-07,0.001918,0.003191,2904.920818,NaN,NaN
2,222-222@65w-qkr-x89,NaN,Fondation Du Centre Hospitalier Regional Du Su...,387 Rue Victoria,Salaberry-de-valleyfield,QC,J6T 1B7,CA,NaN,NaN,2020-09-07T00:00:00-04:00,2020-09-14T00:00:00-04:00,10,7,"[0,1,3,0,0,5,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",CA:24700120,"{""CA:24663033"":4,""CA:24710190"":4,""CA:24700144"":4}","{""CA:24710022"":4,""CA:24700004"":4}","{""CA:24700133"":4}","{""CA"":6}",7127.0,72.0,"{""<5"":0,""5-10"":2,""11-20"":1,""21-60"":2,""61-120"":...","{""McDonald's"":14,""SAQ"":14}","{""Metro"":43,""SAQ"":29,""Shell Oil"":29,""Harnois É...","{""android"":4,""ios"":5}",699.882808,4.788935e-06,6.626535e-06,0.000987,0.001776,839.859370,45.255936,-74.140285
3,222-222@665-55g-b49,NaN,Mohindra B Krishan Dr,959 Chemong Road,Peterborough,ON,K9H 7E3,CA,NaN,NaN,2020-09-07T00:00:00-04:00,2020-09-14T00:00:00-04:00,1,1,"[0,0,0,1,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",CA:35150059,"{""CA:35150123"":4}",{},{},{},NaN,23.0,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":1,""61-120"":...",{},{},"{""android"":0,""ios"":4}",57.637318,2.407436e-07,4.801766e-07,0.000151,0.000214,22.094305,44.322414,-78.335282
4,222-222@665-zbm-zvf,NaN,Bosset Philippe Dr,3 Grenadier Hts,Toronto,ON,M6S 2W5,CA,NaN,NaN,2020-09-07T00:00:00-04:00,2020-09-14T00:00:00-04:00,16,11,"[2,1,1,1,3,7,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0...",CA:35201312,"{""CA:35201487"":4}","{""CA:35200272"":4,""CA:35190123"":4}","{""CA:35201487"":4}","{""CA"":8}",4728.0,26.0,"{""<5"":0,""5-10"":5,""11-20"":1,""21-60"":8,""61-120"":...","{""Circle K Gas"":7,""Esso"":7,""A&W (Canada)"":7,""R...","{""Pet Valu"":18,""Circle K Gas"":9,""Esso"":9,""Petr...","{""android"":7,""ios"":0}",922.197085,3.851898e-06,5.281943e-06,0.002410,0.003427,399.618737,43.641087,-79.468688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624501,zzw-222@63t-wyh-cqz,NaN,Canadian Condominium Institute Windsor Essex C...,1700 Golf View Drive,Windsor,ON,N9J 1Y9,CA,NaN,NaN,2018-12-31T00:00:00-05:00,2019-01-07T00:00:00-05:00,1,1,"[0,0,0,0,0,1,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",CA:35370689,{},"{""CA:35370037"":4}","{""CA:35370704"":4}",{},NaN,97.0,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":0,""61-120"":...",{},{},"{""android"":0,""ios"":0}",154.746264

In [40]:
data = data.dropna(subset=['poi_cbg'])

In [41]:
data['poi_cbg'] = (data['poi_cbg'].str[3:]).astype(int)

In [42]:
data = data.merge(df, on='poi_cbg')

In [9]:
rural = data.loc[data['Type'] == 'Rural']
urban = data.loc[data['Type'] == 'Urban']

In [10]:
rural1 = rural[['date_range_start','region','mobility_score']]

In [18]:
#groupby rural dataframe based on date_range_start and region
rural2 = rural1.groupby(['date_range_start', 'region']).sum()

In [19]:
rural2

mobility_score
date_range_start region                
2018-12-24       AB       146949.418783
                 BC       116871.011501
                 MB        50091.624073
                 NB          384.399374
                 NS       112581.609775
...                                 ...
2022-05-09       NL         3171.903399
                 NS        30652.455672
                 ON       420182.240188
                 QC       327314.107834
                 SK        52694.098196

[1593 rows x 1 columns]

In [12]:
rural1['date_range_start'] = rural1['date_range_start'].str[:10]

In [20]:
rural2.to_csv('df.csv')
rural2 = pd.read_csv('df.csv')

In [21]:
#make date_range_start, mobility_score and region as columns'
rural3 = rural2.pivot_table(index='date_range_start', columns='region', values='mobility_score')

In [22]:
#create an interactive plot which changes with date_range_start. X-axis contains regions and Y-axis contains mobility_score. make the plot more visually appealing
import plotly.express as px
fig = px.bar(rural2, x="region", y="mobility_score", animation_frame="date_range_start", animation_group="region", range_y=[0,1000000])
fig.show()

In [23]:
#make a plot for urban dataframe
urban1 = urban[['date_range_start','region','mobility_score']]
urban1['date_range_start'] = urban1['date_range_start'].str[:10]

urban2 = urban1.groupby(['date_range_start', 'region']).sum()

urban2.to_csv('df2.csv')
urban2 = pd.read_csv('df2.csv')
# urban2 = urban2.pivot_table(in÷dex='date_range_start', columns='region', values='mobility_score')
fig = px.bar(urban2, x="region", y="mobility_score", animation_frame="date_range_start", animation_group="region", range_y=[0,5000000])
fig.show()

In [24]:
data = data[['date_range_start','Type','region','mobility_score']]
data['date_range_start'] = data['date_range_start'].str[:10]
data = data.groupby(['date_range_start', 'Type', 'region']).sum()
data.to_csv('df3.csv')
data = pd.read_csv('df3.csv')

import plotly.express as px
fig = px.bar(data, x="region", y="mobility_score", animation_frame="date_range_start", animation_group="region", range_y=[0,5000000], color='Type')
fig.show()

In [25]:
# data_range_start should be from 2019 to 2021

data = data.loc[data['date_range_start'] >= '2019-06-01']
data = data.loc[data['date_range_start'] <= '2021-06-01']

fig = px.bar(data, x="region", y="mobility_score", animation_frame="date_range_start", animation_group="region", range_y=[0,3000000], color='Type')
fig.show()

In [47]:
import ipywidgets as widgets
from IPython.display import clear_output
import folium
import folium.plugins
warnings.filterwarnings('ignore')

from folium import IFrame

In [ ]:
df3 = data[['latitude','longitude','normalized_visits_by_state_scaling','date_range_start']]
df3['date_range_start'] = df3['date_range_start'].str[:10]
# df3 = df3.groupby(['date_range_start', 'latitude', 'longitude']).sum()
# df3.to_csv('df4.csv')
# df3 = pd.read_csv('df4.csv')
# make a bubble map for normalized_visits_by_state_scaling from df3
fig = px.scatter_mapbox(df3, lat="latitude", lon="longitude", color="normalized_visits_by_state_scaling", size="normalized_visits_by_state_scaling", color_continuous_scale=px.colors.cyclical.IceFire, animation_frame="date_range_start")
fig.show()

In [89]:

def make_map_plot(df_, 
                  plot_limit=15000, 
                  radius_col='normalized_visits_by_state_scaling', 
                  radius_mod=5000, 
                  color='black', 
                  fill_color='red', 
                  fill_opacity = 0.2, 
                  pop_up_col='normalized_visits_by_state_scaling',
                  tooltip = 'Click for more info.', 
                  zoom_start=4,
                  tiles = 'OpenStreetMap', 
                  map_width = 1400,
                  map_height = 800,
                  marker_type= 'circle', 
                  map_plot=None, 
                  verbose=False):
    # Parameters:
      # df_ is a pandas dataframe. It requires a column called "latitude" and a column called "longitude". 
      # radius_col is a column_name or None. If None, every point is given a fixed radius. 
          # Otherwise, the value in the column radius_col is used as the radius. 
      # radius_mod is to scale your radius units to correspond to units on your map.
      # zoom_start is the scale of the map. Larger numbers = higher resolution.
      # color, fill_color, fill_opacity are marker parameters, see others: https://leafletjs.com/reference-1.3.4.html#path 
      # tiles determines the base layer. Open source options include 'OpenStreetMap', 'Stamen Terrain', 'Stamen Toner'
      # map_width and map_height determine the size of the map image (in pixels)
      # marker_type determines what type of marker is being drawn on the map. Options: 'circle' or 'normal'

    # check valid inputs
    valid_inputs = {'marker_type' : {'val' : marker_type, 'valids' : ['circle', 'normal']},
                    'tiles' :  {'val' : tiles, 'valids': ['OpenStreetMap', 'Stamen Terrain', 'Stamen Toner', 'Mapbox Bright', 'Mapbox Control Room']},
                    'radius_col' : {'val' : radius_col, 'valids' : [None] + [col for col in df_.columns if pd.api.types.is_numeric_dtype(df_[col])]}}
    for param, param_valid_dict in valid_inputs.items():
      if(param_valid_dict['val'] not in param_valid_dict['valids']):
        raise Exception("Invalid parameter input for '{0}'. Valid options are {1}. input value was '{2}' .".format(param, param_valid_dict['valids'], param_valid_dict['val']))   

    # create basemap
    if(not map_plot):
      map_plot = folium.Map(width=map_width,
                            height=map_height,
                            location=[df_.latitude.mean(), df_.longitude.mean()],
                            tiles=tiles,
                            zoom_start=zoom_start,
                            control_scale = True)

    # add markers
    counter = 0
    for index, row in df_.iterrows():
        counter+=1
        if(marker_type=='circle'):
          add_circle_marker_to_map(map_plot, row, radius_col, radius_mod, color, fill_color, fill_opacity, pop_up_col, tooltip)
        elif(marker_type=='normal'):
          add_marker_to_map(map_plot, row, pop_up_col, tooltip)
        if(counter>plot_limit): break
    if(verbose): print("Plotted {0} locations".format(counter))
    return(map_plot)


def add_marker_to_map(map_plot, row, pop_up_col, tooltip):
  folium.Marker([row.latitude, row.longitude],
                        popup= row[pop_up_col] if pop_up_col else None,
                        tooltip=tooltip if pop_up_col else None,
                   ).add_to(map_plot)
  return(None)


def add_circle_marker_to_map(map_plot, row, radius_col, radius_mod, color, fill_color, fill_opacity, pop_up_col, tooltip):
  folium.CircleMarker([row.latitude, row.longitude],
                        radius= row[radius_col]/radius_mod if radius_col else 2,
                        color = color,
                        fill_color = fill_color,
                        weight=0.5,
                        fill_opacity= fill_opacity,
                        popup= row[pop_up_col] if pop_up_col else None,
                        tooltip=tooltip if pop_up_col else None,
                   ).add_to(map_plot)
  return(None)

In [90]:

options = list(data['date_range_start'].unique())

date_range = widgets.Dropdown(
    options=options,
    value=options[-1],
    description='Date:',
    disabled=False,
)

output = widgets.Output()

In [91]:
# Function that updates plots with slider value

def update_plot(change):
    with output:
        clear_output(wait=True)
        df_ = data[data['date_range_start'] == change.new]
        # df_ = df_[df_['date_range_start'] <= change.new[0]]

        map = make_map_plot(df_)

        title_html = f'''
                <h3 align="center" style="font-size:16px">{change.new}<b></b></h3>
                 '''
        map.get_root().html.add_child(folium.Element(title_html))

        
        map.save(outfile=f'map{change.new}.html')
        display(map)

In [92]:
#sort dates in options
options.sort()


In [93]:
output = widgets.Output()

with output:
    map = make_map_plot(data[data['date_range_start'] == options[-1]])

    title_html = f'''
            <h3 align="center" style="font-size:16px">{options[-1]}<b></b></h3>
                '''
    map.get_root().html.add_child(folium.Element(title_html))

    display(map)

# Adding slider and reset button in horizontal box
display(widgets.HBox([date_range]))

# Plot update when slider is moved
date_range.observe(update_plot, names='value')

display(output)

Output()

In [52]:
data

,placekey,parent_placekey,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,visitor_home_aggregation,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_week_brand,device_type,normalized_visits_by_state_scaling,normalized_visits_by_total_visits,normalized_visits_by_total_visitors,normalized_visits_by_region_naics_visits,normalized_visits_by_region_naics_visitors,mobility_score,latitude,longitude,Dissemination area (DA) Population,Type
0,222-222@5x6-k2d-nnq,NaN,Sunrise Senior Living,980 Lynn Valley Road,North Vancouver,BC,V7J 3V7,CA,SG_BRAND_516ba83a8efa5bf6bdc8bf2383baf9e1,Sunrise Senior Living,2020-09-07T00:00:00-07:00,2020-09-14T00:00:00-07:00,4,2,"[0,1,0,1,0,1,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",59150148,{},{},"{""CA:59331457"":4}","{""CA"":4}",NaN,382.5,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":0,""61-120"":...",{},{},"{""android"":0,""ios"":0}",290.590368,3.052028e-06,3.187134e-06,0.005457,0.008989,1852.513598,49.332717,-123.047451,400,Urban
1,222-223@5x6-k2d-py9,NaN,North Shore Private Hospital,1070 Lynn Valley Road,North Vancouver,BC,V7J 1Z8,CA,NaN,NaN,2020-07-13T00:00:00-07:00,2020-07-20T00:00:00-07:00,8,7,"[3,2,0,0,1,0,2]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1...",59150148,"{""CA:59152158"":4,""CA:59151770"":4,""CA:59150153"":4}","{""CA:59150206"":4,""CA:59150009"":4}",{},"{""CA"":5}",1075.0,103.5,"{""<5"":1,""5-10"":2,""11-20"":1,""21-60"":0,""61-120"":...","{""BLENZ Coffee"":13,""Esso"":13,""PetSmart"":13,""Br...","{""McDonald's"":43,""Esso"":29,""Petro-Canada"":29,""...","{""android"":9,""ios"":0}",757.090081,6.813706e-06,1.480191e-05,0.001358,0.002683,1305.980390,49.334061,-123.044528,400,Urban
2,222-223@5x6-k2d-py9,NaN,North Shore Private Hospital,1070 Lynn Valley Road,North Vancouver,BC,V7J 1Z8,CA,NaN,NaN,2020-03-09T00:00:00-07:00,2020-03-16T00:00:00-07:00,30,14,"[6,3,4,4,8,2,3]","[0,0,0,0,0,0,0,1,1,2,0,0,0,1,0,0,0,1,0,0,0,0,0...",59150148,"{""CA:59153240"":4,""CA:59150094"":4}",{},"{""CA:59150114"":4,""CA:59150182"":4}","{""CA"":14}",3689.0,86.0,"{""<5"":1,""5-10"":4,""11-20"":3,""21-60"":3,""61-120"":...","{""McDonald's"":9,""Bean Around the World Coffees...","{""BLENZ Coffee"":21,""McDonald's"":21,""Walmart"":1...","{""android"":6,""ios"":7}",1963.785648,1.609917e-05,2.179599e-05,0.002738,0.005265,2814.759429,49.334061,-123.044528,400,Urban
3,222-222@5x6-k2d-nnq,NaN,Sunrise Senior Living,980 Lynn Valley Rd,North Vancouver,BC,V7J 3V7,CA,SG_BRAND_516ba83a8efa5bf6bdc8bf2383baf9e1,Sunrise Senior Living,2022-05-02T00:00:00-07:00,2022-05-09T00:00:00-07:00,8,4,"[2,1,1,1,1,1,1]","[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0...",59150148,"{""CA:59150079"":4,""CA:59152809"":4}","{""CA:59150333"":4}","{""CA:59150079"":4,""CA:59152809"":4}","{""CA"":4}",NaN,484.0,"{""<5"":0,""5-10"":0,""11-20"":1,""21-60"":2,""61-120"":...",{},{},"{""android"":0,""ios"":4}",382.106389,7.588534e-06,7.946899e-06,0.205128,0.444444,3082.324872,49.332717,-123.047451,400,Urban
4,222-222@5x6-k2d-nnq,NaN,Sunrise Senior Living,980 Lynn Valley Road,North Vancouver,BC,V7J 3V7,CA,SG_BRAND_516ba83a8efa5bf6bdc8bf2383baf9e1,Sunrise Senior Living,2020-07-13T00:00:00-07:00,2020-07-20T00:00:00-07:00,19,9,"[1,5,3,2,2,5,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0...",59150148,"{""CA:59153397"":4,""CA:59154007"":4}","{""CA:59150109"":4,""CA:59150028"":4}","{""CA:59150148"":4,""CA:59153397"":4}","{""CA"":7}",2755.0,163.0,"{""<5"":0,""5-10"":1,""11-20"":1,""21-60"":3,""61-120"":...","{""Shoppers Drug Mart"":7,""Second Cup"":7,""Esso"":...","{""Esso"":44,""RONA"":22,""Starbucks"":22,""Second Cu...","{""android"":7,""ios"":6}",1798.088942,1.618255e-05,1.903102e-05,0.027576,0.053073,4884.808293,49.332717,-123.047451,400,Urban
...,...,...,...,...,...,...,...,...,...,..